In [6]:
import tensorflow as tf
import tensorflow_hub as hub

print(tf.__version__)
print(hub.__version__)


2.10.1
0.14.0


In [5]:
import tensorflow as tf
import tensorflow_hub as hub
from flask import Flask, request, render_template_string

app = Flask(__name__)

# Load the BERT model
model = tf.keras.models.load_model(r'C:\Users\akash\Python_DL\PROJECT_DL\email_classification_model.h5', custom_objects={'KerasLayer': hub.KerasLayer})

# Preprocess function
def preprocess_text(text):
    # Load the tokenizer from the BERT encoder
    tokenizer = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

    # Tokenize the input text
    tokenized_text = tokenizer([text], max_seq_length=128)

    # Convert tokenized text to input format expected by BERT
    input_ids = tokenized_text['input_word_ids']
    input_mask = tokenized_text['input_mask']
    input_type_ids = tokenized_text['input_type_ids']

    return input_ids, input_mask, input_type_ids

@app.route('/', methods=['GET', 'POST'])
def index():
    result = None
    if request.method == 'POST':
        sentence = request.form['sentence']
        input_ids, input_mask, input_type_ids = preprocess_text(sentence)
        prediction = model.predict([input_ids, input_mask, input_type_ids])
        result = "Spam" if prediction[0][0] >= 0.5 else "Ham"
    return render_template_string(open('index.html').read(), result=result)

if __name__ == '__main__':
    app.run(debug=True)


FileNotFoundError: Op type not registered 'CaseFoldUTF8' in binary running on MSI. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.
 You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` to the io_device such as '/job:localhost'.

In [ ]:
import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle

app = Flask(__name__)
model = pickle.load(open('model.pkl', 'rb'))

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    int_features = [int(x) for x in request.form.values()]
    final_features = [np.array(int_features)]
    prediction = model.predict(final_features)

    output = round(prediction[0], 2)

    return render_template('index.html', prediction_text='Employee Salary should be $ {}'.format(output))

@app.route('/predict_api',methods=['POST'])
def predict_api():
    '''
    For direct API calls trought request
    '''
    data = request.get_json(force=True)
    prediction = model.predict([np.array(list(data.values()))])

    output = prediction[0]
    return jsonify(output)

if __name__ == "__main__":
    app.run(debug=True)

In [7]:
import streamlit as st
from streamlit_jupyter import streamlit_patcher
streamlit_patcher.jupyter()

ModuleNotFoundError: No module named 'streamlit'

In [5]:
# !pip install streamlit
!pip install --upgrade protobuf


In [8]:
import streamlit as st
import pickle
# import string
# from nltk.corpus import stopwords
# import nltk
# from nltk.stem.porter import PorterStemmer

# ps = PorterStemmer()


# def transform_text(text):
#     text = text.lower()
#     text = nltk.word_tokenize(text)

#     y = []
#     for i in text:
#         if i.isalnum():
#             y.append(i)

#     text = y[:]
#     y.clear()

#     for i in text:
#         if i not in stopwords.words('english') and i not in string.punctuation:
#             y.append(i)

#     text = y[:]
#     y.clear()

#     for i in text:
#         y.append(ps.stem(i))

#     return " ".join(y)

# tfidf = pickle.load(open('vectorizer.pkl','rb'))
model = pickle.load(open(r'C:\Users\akash\Python_DL\PROJECT_DL\model.pkl','rb'))

st.title("Email/SMS Spam Classifier")

input_sms = st.text_area("Enter the message")

if st.button('Predict'):

#     # 1. preprocess
#     transformed_sms = transform_text(input_sms)
#     # 2. vectorize
#     vector_input = tfidf.transform([transformed_sms])
    # 3. predict
    result = model.predict(vector_input)[0]
    # 4. Display
    if result == 1:
        st.header("Spam")
    else:
        st.header("Not Spam")

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates